# # 提取航班数据相关特征  FlightDataFeature.csv
1.计划起飞时间/计划到达时间、实际起飞时间/实际到达时间，转换为日期格式，并分别提出日期/时刻/月份/季节
2.计划飞行时间/实际飞行时间
3.起飞延误时间/到达延误时间
4.根据航班提取： 航空公司及其性质（返程、去程、国内/国际航班）
5.根据航班提取：本航班的平均延误时间，延误时间标准差，最大延误时间，延误时间中位数
6.前序航班特征：起飞间隔/前序延误

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 航班数据
flight_data = pd.read_table('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/train/flightData.csv',sep=',',encoding='gb2312')
flight_data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常


In [2]:
#将计划时间转换成为日期格式
flight_data['计划起飞时间1'] = pd.to_datetime(flight_data['计划起飞时间'], unit = 's', utc = True)
flight_data['计划到达时间1'] = pd.to_datetime(flight_data['计划到达时间'], unit = 's', utc = True)

#计划飞行时间
flight_data['计划飞行时间'] = flight_data['计划到达时间1'] - flight_data['计划起飞时间1']
flight_data['计划飞行时间'] = flight_data['计划飞行时间'].apply(lambda x : x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
#转换为小时
flight_data['计划飞行时间'] = flight_data['计划飞行时间'] / 3600
flight_data

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,计划到达时间1,计划飞行时间
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常,2016-01-26 12:00:00,2016-01-26 14:05:00,2.083333
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常,2016-01-14 08:40:00,2016-01-14 10:25:00,1.750000
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常,2016-01-26 09:35:00,2016-01-26 11:25:00,1.833333
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常,2016-01-06 22:50:00,2016-01-07 01:45:00,2.916667
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常,2016-01-10 04:10:00,2016-01-10 06:20:00,2.166667
5,DLC,NNG,ZH953Z,1452385800,1452401700,NaN,NaN,6.0,取消,2016-01-10 00:30:00,2016-01-10 04:55:00,4.416667
6,HGH,SZX,CZ6327,1452591900,1452600900,NaN,NaN,NaN,取消,2016-01-12 09:45:00,2016-01-12 12:15:00,2.500000
7,NKG,HAK,GS6538,1452999900,1453010100,1.453005e+09,1.453014e+09,7.0,正常,2016-01-17 03:05:00,2016-01-17 05:55:00,2.833333
8,LZO,PEK,MU5709,1452910200,1452919800,1.452910e+09,1.452918e+09,8.0,正常,2016-01-16 02:10:00,2016-01-16 04:50:00,2.666667
9,WEH,HGH,CA1788,1453708200,1453715100,1.453708e+09,1.453714e+09,9.0,正常,2016-01-25 07:50:00,2016-01-25 09:45:00,1.916667


In [3]:
#将实际时间转换成为日期格式
flight_data['实际起飞时间1'] = pd.to_datetime(flight_data['实际起飞时间'], unit = 's', utc = True)
flight_data['实际到达时间1'] = pd.to_datetime(flight_data['实际到达时间'], unit = 's', utc = True)
#实际飞行时间
flight_data['实际飞行时间'] = flight_data['实际到达时间1'] - flight_data['实际起飞时间1']
flight_data['实际飞行时间'] = flight_data['实际飞行时间'].apply(lambda x : x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
#转换为小时
flight_data['实际飞行时间'] = flight_data['实际飞行时间'] / 3600
flight_data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,计划到达时间1,计划飞行时间,实际起飞时间1,实际到达时间1,实际飞行时间
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常,2016-01-26 12:00:00,2016-01-26 14:05:00,2.083333,2016-01-26 12:58:00,2016-01-26 14:43:00,1.750000
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常,2016-01-14 08:40:00,2016-01-14 10:25:00,1.750000,2016-01-14 09:10:00,2016-01-14 10:39:00,1.483333
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常,2016-01-26 09:35:00,2016-01-26 11:25:00,1.833333,2016-01-26 09:59:00,2016-01-26 11:14:00,1.250000
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常,2016-01-06 22:50:00,2016-01-07 01:45:00,2.916667,2016-01-06 23:04:00,2016-01-07 01:24:00,2.333333
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常,2016-01-10 04:10:00,2016-01-10 06:20:00,2.166667,2016-01-10 04:25:00,2016-01-10 05:39:00,1.233333


In [4]:
#起飞延误时间 = 实际起飞时间 - 计划起飞时间
flight_data['起飞延误时间'] = flight_data['实际起飞时间1'] - flight_data['计划起飞时间1']
flight_data['起飞延误时间'] = flight_data['起飞延误时间'].apply(lambda x : x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
#转换为小时
flight_data['起飞延误时间'] = flight_data['起飞延误时间'] / 3600

#到达延误时间 = 实际到达时间 - 计划到达时间

flight_data['到达延误时间'] = flight_data['实际到达时间1'] - flight_data['计划到达时间1']
flight_data['到达延误时间'] = flight_data['到达延误时间'].apply(lambda x : x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
#转换为小时
flight_data['到达延误时间'] = flight_data['到达延误时间'] / 3600

In [5]:
#将航班已经取消的直接设置为延误10小时以上
flight_data['起飞延误时间'] = np.where(flight_data['航班是否取消'] == '取消',10,flight_data['起飞延误时间'])
flight_data['到达延误时间'] = np.where(flight_data['航班是否取消'] == '取消',10,flight_data['到达延误时间'])

In [6]:
#细分时间段
flight_data['计划起飞日期'] = flight_data['计划起飞时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
flight_data['计划起飞时刻'] = flight_data['计划起飞时间1'].apply(lambda x:x.strftime('%H') if not(pd.isnull(x)) else None)
flight_data['航班月份'] = flight_data['计划起飞时间1'].apply(lambda x:int(x.strftime('%m')) if not(pd.isnull(x)) else None)

flight_data['计划到达日期'] = flight_data['计划到达时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
flight_data['计划到达时刻'] = flight_data['计划到达时间1'].apply(lambda x:x.strftime('%H') if not(pd.isnull(x)) else None)
flight_data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,实际起飞时间1,实际到达时间1,实际飞行时间,起飞延误时间,到达延误时间,计划起飞日期,计划起飞时刻,航班月份,计划到达日期,计划到达时刻
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常,2016-01-26 12:00:00,...,2016-01-26 12:58:00,2016-01-26 14:43:00,1.750000,0.966667,0.633333,2016-01-26,12,1,2016-01-26,14
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常,2016-01-14 08:40:00,...,2016-01-14 09:10:00,2016-01-14 10:39:00,1.483333,0.500000,0.233333,2016-01-14,08,1,2016-01-14,10
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常,2016-01-26 09:35:00,...,2016-01-26 09:59:00,2016-01-26 11:14:00,1.250000,0.400000,-0.183333,2016-01-26,09,1,2016-01-26,11
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常,2016-01-06 22:50:00,...,2016-01-06 23:04:00,2016-01-07 01:24:00,2.333333,0.233333,-0.350000,2016-01-06,22,1,2016-01-07,01
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常,2016-01-10 04:10:00,...,2016-01-10 04:25:00,2016-01-10 05:39:00,1.233333,0.250000,-0.683333,2016-01-10,04,1,2016-01-10,06


In [14]:
#航空公司
flight_data['航空公司'] = flight_data['航班编号'].apply(lambda x : x[0:2])

# fileName_overdue = open(r'E:/季节表.txt', encoding="utf-8")
# 季节表 = pd.read_csv(fileName_overdue, header=None, names=["航班月份", "航班季节"])

季节表 = pd.read_csv('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/季节表.txt',sep='\s+')
#航班季节
flight_data = pd.merge(flight_data, 季节表, how = 'left',on='航班月份')
flight_data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,实际飞行时间,起飞延误时间,到达延误时间,计划起飞日期,计划起飞时刻,航班月份,计划到达日期,计划到达时刻,航空公司,航班季节
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常,2016-01-26 12:00:00,...,1.750000,0.966667,0.633333,2016-01-26,12,1,2016-01-26,14,CZ,NaN
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常,2016-01-14 08:40:00,...,1.483333,0.500000,0.233333,2016-01-14,08,1,2016-01-14,10,FM,NaN
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常,2016-01-26 09:35:00,...,1.250000,0.400000,-0.183333,2016-01-26,09,1,2016-01-26,11,ZH,NaN
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常,2016-01-06 22:50:00,...,2.333333,0.233333,-0.350000,2016-01-06,22,1,2016-01-07,01,9C,NaN
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常,2016-01-10 04:10:00,...,1.233333,0.250000,-0.683333,2016-01-10,04,1,2016-01-10,06,TZ,NaN


In [15]:
flight_data['是否为去程航班'] = flight_data['航班编号'].apply(lambda x : x[-1])
flight_data['是否为去程航班'] = flight_data['是否为去程航班'].apply(lambda x : 2 if 'A'<= x <= 'Z' else (1 if int(x) % 2 == 0 else 0))
#航班构造，一般末位数为单数是去程航班，偶数是回程航班，字母是补班航班
#是否为去程航班，1 表示是，0表示否， 2表示补班航班

flight_data['是否为去程航班'] = flight_data['是否为去程航班'].apply(lambda x : 1 if int(x) == 0 else 0)

#是否补班航班 ，1表示是， 0表示否
flight_data['是否是补班航班'] = flight_data['是否为去程航班'].apply(lambda x : 1 if int(x) == 2 else 0)
#航班性质，国内航班1，国际航班0
flight_data['是否为国内航班'] = flight_data['航班编号'].apply(lambda x : 1 if len(x) == 6 else 0)
flight_data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,计划起飞日期,计划起飞时刻,航班月份,计划到达日期,计划到达时刻,航空公司,航班季节,是否为去程航班,是否是补班航班,是否为国内航班
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常,2016-01-26 12:00:00,...,2016-01-26,12,1,2016-01-26,14,CZ,NaN,0,0,1
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常,2016-01-14 08:40:00,...,2016-01-14,08,1,2016-01-14,10,FM,NaN,1,0,1
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常,2016-01-26 09:35:00,...,2016-01-26,09,1,2016-01-26,11,ZH,NaN,1,0,1
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常,2016-01-06 22:50:00,...,2016-01-06,22,1,2016-01-07,01,9C,NaN,1,0,1
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常,2016-01-10 04:10:00,...,2016-01-10,04,1,2016-01-10,06,TZ,NaN,1,0,0


# # 前序航班特征提取

In [16]:
#飞机编号有缺失值的统一编号为0
flight_data['飞机编号'] = flight_data['飞机编号'].fillna(0)
flight_data['前序延误'] = pd.Series()
flight_data['起飞间隔'] = pd.Series()

#根据航班的飞机编号进行分组
#前序延误 = 前序航班 实际到达时间 - 计划到达时间
#起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
 
grouped = flight_data.groupby(flight_data['飞机编号'])
chunks = []
for name, group in grouped:
    
    group = group.sort_values('计划起飞时间')
    a = pd.to_datetime(group['计划起飞时间'], unit = 's', utc = True)[1: ].reset_index(drop = True)
    b = pd.to_datetime(group['实际到达时间'], unit='s', utc=True)[0:len(group)-1].reset_index(drop=True)
    group['起飞间隔'][1: ] = a - b
    group['起飞间隔'] = group['起飞间隔'].apply(lambda x : x.days * 86400 + x.seconds if not (pd.isnull(x)) else None)
    group['起飞间隔'] = group['起飞间隔'] / 3600
    group['前序延误'][1: ] = group['到达延误时间'][0:len(group)-1]
#     group['前序航班编号'] = 
    chunks.append(group)
flight_data = pd.concat(chunks, ignore_index = True)

flight_data.head()    
    

D:\anaconda3\anaconda3_install\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda3\anaconda3_install\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,航班月份,计划到达日期,计划到达时刻,航空公司,航班季节,是否为去程航班,是否是补班航班,是否为国内航班,前序延误,起飞间隔
0,JHG,KMG,8L9903A,1430410500,1430414400,1.430414e+09,1.430417e+09,0.0,正常,2015-04-30 16:15:00,...,4,2015-04-30,17,8L,NaN,0,0,0,NaN,NaN
1,JHG,KMG,8L9905A,1430415000,1430418000,1.430415e+09,1.430418e+09,0.0,正常,2015-04-30 17:30:00,...,4,2015-04-30,18,8L,NaN,0,0,0,0.683333,-0.516667
2,PEK,HGH,GJ8888,1430430900,1430439000,1.430432e+09,1.430438e+09,0.0,正常,2015-04-30 21:55:00,...,4,2015-05-01,00,GJ,NaN,0,0,1,0.100000,3.48333
3,PEK,DQA,CA1683,1430432100,1430439900,NaN,NaN,0.0,取消,2015-04-30 22:15:00,...,4,2015-05-01,00,CA,NaN,1,0,1,-0.283333,-1.63333
4,HAK,SZX,DZ6262,1430433000,1430437200,1.430434e+09,1.430437e+09,0.0,正常,2015-04-30 22:30:00,...,4,2015-04-30,23,DZ,NaN,0,0,1,10.000000,NaN


In [17]:
#添加本航班的特征
#本航班的平均延误时间，延误时间标准差，最大延误时间，延误时间中位数
flight_feature = []
for flight in flight_data.groupby([u'航班编号', u'出发机场']):
    describe = flight[1]['起飞延误时间'].describe()
    if len(flight[1]) > 1:
        flight_feature.append(
                [flight[0][0], flight[0][1], describe['mean'], describe['std'], describe['max'], describe['50%']])
    else:
        flight_feature.append([flight[0][0], flight[0][1], describe['mean'], 0, describe['max'], describe['50%']])

df_flight_feature = pd.DataFrame(flight_feature,
                                     columns=[u'航班编号', u'出发机场', u'平均延误时间', u'延误时间标准差', u'最大延误时间', u'延误时间中位数'])
df_flight_feature.head()

,航班编号,出发机场,平均延误时间,延误时间标准差,最大延误时间,延误时间中位数
0,3K243,SIN,0.266667,0.000000,0.266667,0.266667
1,3K515,SIN,0.233333,0.000000,0.233333,0.233333
2,3K601,SIN,0.241667,0.011785,0.250000,0.241667
3,3U1012,CKG,10.000000,0.000000,10.000000,10.000000
4,3U2026,CKG,10.000000,0.000000,10.000000,10.000000


In [18]:
flight_data = pd.merge(flight_data, df_flight_feature, how = 'left', on = [u'航班编号', u'出发机场'])

# 特情数据处理 Flight_SpecInfoDataFeature.csv
1.根据特情数据提取：特情开始/结束时刻 2.出发/到达机场是否有特情

In [19]:
SpecialInfo_data = pd.read_excel('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/train/specialInfoData.xlsx')
SpecialInfo_data.head()

,特情机场,收集时间,开始时间,结束时间,特情内容
0,csx,2015-04-30 19:52:45Z,2015-05-01 12:00:00Z,2015-05-01 16:00:00Z,长沙机场航班延误黄色预警提示：预计明日（5月1日）12:00-16:00长沙黄花机场受雷雨天...
1,wuh,2015-05-01 08:13:39Z,2015-05-01 08:00:00Z,2015-05-01 12:00:00Z,目前，8时-12时雷雨，期间进出港航班可能受到影响。
2,khn,2015-05-01 08:14:18Z,2015-05-01 14:00:00Z,2015-05-01 17:00:00Z,预计，14时-17时小雷雨，期间进出港航班可能受到影响。
3,swa,2015-05-01 08:24:28Z,2015-05-01 13:00:00Z,2015-05-01 17:00:00Z,预计，13时-17时雷雨，期间进出港航班可能受到影响。
4,zha,2015-05-01 08:24:56Z,2015-05-01 13:00:00Z,2015-05-01 17:00:00Z,预计，13时-17时雷雨，期间进出港航班可能受到影响。


In [20]:
# 将机场的代码转换为大写
for i in SpecialInfo_data.index:
    s = str(SpecialInfo_data['特情机场'][i])
    SpecialInfo_data['特情机场'][i] = s.upper()

In [21]:
#  提取出特情发生的开始结束日期和时刻
SpecialInfo_data['开始日期'] = SpecialInfo_data['开始时间'].apply(lambda x :str(x).split(' ')[0])
SpecialInfo_data['开始时刻'] = pd.Series()
SpecialInfo_data['结束时刻'] = pd.Series()

In [22]:
#提取时刻
for i in SpecialInfo_data.index:
    if not(pd.isnull(SpecialInfo_data['开始时间'][i])):
        SpecialInfo_data['开始时刻'][i] = int(str(SpecialInfo_data['开始时间'][i]).split(' ')[1][:2])
    else :
        SpecialInfo_data['开始时刻'][i] = np.NaN
    if not(pd.isnull(SpecialInfo_data['结束时间'][i])):
        SpecialInfo_data['结束时刻'][i] = int(str(SpecialInfo_data['结束时间'][i]).split(' ')[1][:2])
    else :
        SpecialInfo_data['结束时刻'][i] = np.NaN

D:\anaconda3\anaconda3_install\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\anaconda3\anaconda3_install\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda3\anaconda3_install\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\anaconda3\anaconda3_install\lib\sit

In [23]:
# 去重，避免左连接时发生多余样本
SpecialInfo_data = SpecialInfo_data.drop_duplicates(['特情机场','开始日期'])

In [31]:
# 到达城市特情
flight_data = pd.merge(flight_data,SpecialInfo_data,left_on=['到达机场','计划到达日期'],right_on=['特情机场','开始日期'],how='left',sort=False)
# 落在相应的时间段即为有特情
flight_data['到达特情'] = np.where((flight_data['计划到达时刻'].astype('float') >= flight_data['开始时刻'].astype('float')) &
                               (flight_data['计划到达时刻'].astype('float') <=  flight_data['结束时刻'].astype('float')),1,0)


In [33]:

del(flight_data['特情机场'])
del(flight_data['开始日期'] )
del(flight_data['开始时刻'])
del(flight_data['结束时刻'] )

In [34]:
# 出发城市特情
flight_data = pd.merge(flight_data,SpecialInfo_data,left_on=['出发机场','计划起飞日期'],right_on=['特情机场','开始日期'],how='left',sort=False)
# 落在相应的时间段即为有特情


In [36]:
flight_data['出发特情'] = np.where((flight_data['计划起飞时刻'].astype('float') >= flight_data['开始时刻'].astype('float')) &
                               (flight_data['计划起飞时刻'].astype('float') <= flight_data['结束时刻'].astype('float')),1,0)

In [37]:
del(flight_data['收集时间_y'])
del(flight_data['开始时间_y'] )
del(flight_data['结束时间_y'])
del(flight_data['特情内容_y'] )
del(flight_data['收集时间_x'])
del(flight_data['特情内容_x'] )
flight_data = flight_data.rename(columns={'开始时间_x':'开始时间','结束时间_x':'结束时间'})

In [38]:
flight_data.columns

Index(['出发机场', '到达机场', '航班编号', '计划起飞时间', '计划到达时间', '实际起飞时间', '实际到达时间', '飞机编号',
       '航班是否取消', '计划起飞时间1', '计划到达时间1', '计划飞行时间', '实际起飞时间1', '实际到达时间1',
       '实际飞行时间', '起飞延误时间', '到达延误时间', '计划起飞日期', '计划起飞时刻', '航班月份', '计划到达日期',
       '计划到达时刻', '航空公司', '航班季节', '是否为去程航班', '是否是补班航班', '是否为国内航班', '前序延误',
       '起飞间隔', '平均延误时间', '延误时间标准差', '最大延误时间', '延误时间中位数', '开始时间', '结束时间',
       '到达特情', '特情机场', '开始日期', '开始时刻', '结束时刻', '出发特情'],
      dtype='object')

In [ ]:
flight_data.to_csv("/Users/tuyu/000000毕业设计/Flight_Delay/Flight_SpecInfoDataFeature.csv",index=None,encoding="gb2312")